In [ ]:
import torch
import torchvision.datasets as datasets
import os
import foolbox
import torchvision.models as models
import numpy as np
import cv2
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

BATCH_SIZE = 64
datapath = '/home/user/datasets/ImageNet/'
traindir = os.path.join(datapath, 'train')
labeldir = '/home/user/datasets/ImageNet/class_to_idx.txt'


train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
#         transforms.Resize(224),
       transforms.CenterCrop(224),
        transforms.ToTensor(),
   #      transforms.Normalize(mean=[0.485, 0.456, 0.406],
    #                 	     std=[0.229, 0.224, 0.225])
    ])
)


# train_loader = torch.utils.data.DataLoader(
#     train_dataset, batch_size=BATCH_SIZE, shuffle=False,
#     num_workers=1, pin_memory=True, sampler=None)

In [ ]:
import foolbox

In [ ]:
resnet101 = models.resnet101(pretrained=True).eval()
if torch.cuda.is_available():
    resnet101 = resnet101.cuda()
else:
    print('===============')
mean = np.array([0.485, 0.456, 0.406]).reshape((3, 1, 1))
std = np.array([0.229, 0.224, 0.225]).reshape((3, 1, 1))
fmodel = foolbox.models.PyTorchModel(
    resnet101, bounds=(0, 1), num_classes=1000, preprocessing=(mean, std))

In [ ]:
from scipy import ndimage
import tensorflow as tf
from abdm.abdm import ABDM
transform = transforms.Compose([transforms.ToTensor()])

In [ ]:
img_id=[]        #ori images ID list
img_ori=[]       #ori images list
img_adv=[]       #adv images list
img_label=[]     #ori images labels list
stn_image=[]     #stn images list  

wrong_oriimg=0
right_advimg=0
wrong_advimg=0
right_stnimg=0
wrong_stnimg=0
list=[1007112, 1007113,1007114, 1007115, 1007116,1007117,1007118,1007119,1007120,1007121,1007122,1007123, 1007126,1007127, 1007128, 1007129, 1007132, 1007138, 1007143, 1007145, 1007147, 1007153, 1007167, 1007171,1007172,1007173,1007174,1007175,1007176, 1007177,1007178, 1007179, 1007185, 1007187, 1007194, 1007196, 1007197, 1007198, 1007202, 1007204, 1007207, 1007212, 1007213, 1007215, 1007221, 1007223, 1007226, 1007229, 1007232, 1007234, 1007236, 1007242, 1007243, 1007245, 1007247, 1007250, 1007258, 1007261, 1007262, 1007266, 1007271, 1007273, 1007276, 1007278, 1007280, 1007283, 1007285, 1007289, 1007294, 1007300, 1007307, 1007310, 1007312, 1007313, 1007315, 1007317, 1007321, 1007323, 1007327, 1007330, 1007332, 1007336, 1007337, 1007341, 1007342, 1007347, 1007359, 1007360, 1007363, 1007366, 1007369, 1007370, 1007374, 1007378, 1007380, 1007382, 1007386, 1007389, 1007392, 1007400 ]

for num in list:
    image, target = train_dataset[num]
    image= np.array(image)
    print('predicted class', np.argmax(fmodel.predictions(image)),', ground truth class',target)
    tempclass1=str(np.argmax(fmodel.predictions(image)))
    tempclass2=str(target)
    if(tempclass1!=tempclass2):
        wrong_oriimg=wrong_oriimg+1
        continue
        
    #dp_attack = foolbox.attacks.FGSM(fmodel)
    dp_attack = foolbox.attacks.deepfool.DeepFoolAttack(fmodel, distance=foolbox.distances.Linfinity)
    #dp_attack = foolbox.attacks.PGD(fmodel, distance=foolbox.distances.Linfinity)
    adversarial = dp_attack(image, target)
    try:
        print('adversarial class', np.argmax(fmodel.predictions(adversarial)))
    except: 
        wrong_advimg=wrong_advimg+1
        print('error')
        continue
    else:
        right_advimg=right_advimg+1
        print('adversarial class', np.argmax(fmodel.predictions(adversarial)))

    
    #===============abdm start (0.0)=========================================
    im=adversarial
    im = transform(im).numpy()
    im = transform(im).numpy()
    image_show=im
    #im=im.resize(3,224,224)
    print('ori image shape is :',im.shape)
    print("===========================================================")
    im = im.reshape(1, 224, 224, 3)
    im = im.astype('float32')
    #print('img-over')
    out_size = (224, 224)
    batch = np.append(im, im, axis=0)
    batch = np.append(batch, im, axis=0)
    num_batch = 3 
    x = tf.placeholder(tf.float32, [None, 224, 224, 3])
    x = tf.cast(batch, 'float32')
    print('begin---')
    with tf.variable_scope('abdm_0'):
        n_fc = 6
        w_fc1 = tf.Variable(tf.Variable(tf.zeros([224 * 224 * 3, n_fc]), name='W_fc1'))
        initial = np.array([[0.5, 0, 0], [0, 0.5, 0]])
        initial = initial.astype('float32')
        initial = initial.flatten()  
        b_fc1 = tf.Variable(initial_value=initial, name='b_fc1')   
        h_fc1 = tf.matmul(tf.zeros([num_batch, 224 * 224 * 3]), w_fc1) + b_fc1
        print(x, h_fc1, out_size)
        h_trans = ABDM(x, h_fc1, out_size)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    y = sess.run(h_trans, feed_dict={x: batch})
    stnimg_temp=transform(y[0]).numpy() 
    
    adv_class=str(np.argmax(fmodel.predictions(stnimg_temp)))
    orilabel=str(target)
    print('adversarial class', np.argmax(fmodel.predictions(stnimg_temp)))
    print('ori class', orilabel)
    if(adv_class==orilabel):
        # put images and labels into list
        img_ori.append(image)
        img_adv.append(adversarial)
        img_label.append(target)
        img_id.append(num)
        stn_image.append(stnimg_temp)
        print(len(img_id))
        right_stnimg=right_stnimg+1
    else:
        print('can not use this img')
        wrong_stnimg=wrong_stnimg+1
        continue

        
ori_right=(100-wrong_oriimg)/100
adv_right=(wrong_oriimg+wrong_advimg)/100
stn_right=right_stnimg/100
stn_right2=right_stnimg/(right_stnimg+wrong_stnimg)

print('clean image Accuracy:  %.2f%%' % (ori_right * 100))
print('adv image Accuracy:  %.2f%%' % (adv_right * 100))
print('stn image Accuracy:  %.2f%%' % (stn_right * 100 ))      
print('stn image Accuracy:  %.2f%%' % (stn_right2 * 100 ))   